<!--- Licensed to the Apache Software Foundation (ASF) under one -->
<!--- or more contributor license agreements.  See the NOTICE file -->
<!--- distributed with this work for additional information -->
<!--- regarding copyright ownership.  The ASF licenses this file -->
<!--- to you under the Apache License, Version 2.0 (the -->
<!--- "License"); you may not use this file except in compliance -->
<!--- with the License.  You may obtain a copy of the License at -->

<!---   http://www.apache.org/licenses/LICENSE-2.0 -->

<!--- Unless required by applicable law or agreed to in writing, -->
<!--- software distributed under the License is distributed on an -->
<!--- "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY -->
<!--- KIND, either express or implied.  See the License for the -->
<!--- specific language governing permissions and limitations -->
<!--- under the License. -->

# NDArray - Imperative tensor operations on CPU/GPU

In _MXNet_, `NDArray` is the core data structure for all mathematical
computations.  An `NDArray` represents a multidimensional, fixed-size homogenous
array.  If you're familiar with the scientific computing python package
[NumPy](http://www.numpy.org/), you might notice that `mxnet.ndarray` is similar
to `numpy.ndarray`.  Like the corresponding NumPy data structure, MXNet's
`NDArray` enables imperative computation.

So you might wonder, why not just use NumPy?  MXNet offers two compelling
advantages.  First, MXNet's `NDArray` supports fast execution on a wide range of
hardware configurations, including CPU, GPU, and multi-GPU machines.  _MXNet_
also scales to distributed systems in the cloud.  Second, MXNet's `NDArray`
executes code lazily, allowing it to automatically parallelize multiple
operations across the available hardware.

An `NDArray` is a multidimensional array of numbers with the same type.  We
could represent the coordinates of a point in 3D space, e.g. `[2, 1, 6]` as a 1D
array with shape (3).  Similarly, we could represent a 2D array.  Below, we
present an array with length 2 along the first axis and length 3 along the
second axis.
```
[[0, 1, 2]
 [3, 4, 5]]
```
Note that here the use of "dimension" is overloaded.  When we say a 2D array, we
mean an array with 2 axes, not an array with two components.

Each NDArray supports some important attributes that you'll often want to query:

- **ndarray.shape**: The dimensions of the array. It is a tuple of integers
  indicating the length of the array along each axis. For a matrix with `n` rows
  and `m` columns, its `shape` will be `(n, m)`.
- **ndarray.dtype**: A `numpy` _type_ object describing the type of its
  elements.
- **ndarray.size**: The total number of components in the array - equal to the
  product of the components of its `shape`
- **ndarray.context**: The device on which this array is stored, e.g. `cpu()` or
  `gpu(1)`.

## Prerequisites

To complete this tutorial, we need:

- MXNet. See the instructions for your operating system in [Setup and Installation](http://mxnet.io/install/index.html)
- [Jupyter](http://jupyter.org/)
    ```
    pip install jupyter
    ```
- GPUs - A section of this tutorial uses GPUs. If you don't have GPUs on your
machine, simply set the variable gpu_device (set in the GPUs section of this 
tutorial) to mx.cpu().

## Array Creation

There are a few different ways to create an `NDArray`.

* We can create an NDArray from a regular Python list or tuple by using the `array` function:

In [1]:
import mxnet as mx
# create a 1-dimensional array with a python list
# a = mx.nd.array([1,2,3])
a=mx.nd.array([1,2,3])
# create a 2-dimensional array with a nested python list
b = mx.nd.array([[1,2,3], [2,3,4]])
{'a.shape':a.shape, 'b.shape':b.shape}

{'a.shape': (3,), 'b.shape': (2, 3)}

* We can also create an MXNet NDArray from a `numpy.ndarray` object:

In [2]:
import numpy as np
import math
c = np.arange(15).reshape(3,5)
# create a 2-dimensional array from a numpy.ndarray object
a = mx.nd.array(c)
{'a.shape':a.shape}

{'a.shape': (3, 5)}

We can specify the element type with the option `dtype`, which accepts a numpy
type. By default, `float32` is used:

In [3]:
# float32 is used by default
a = mx.nd.array([1,2,3])
# create an int32 array
b = mx.nd.array([1,2,3], dtype=np.int32)
# create a 16-bit float array
c = mx.nd.array([1.2, 2.3], dtype=np.float16)
(a.dtype, b.dtype, c.dtype)

(numpy.float32, numpy.int32, numpy.float16)

If we know the size of the desired NDArray, but not the element values, MXNet
offers several functions to create arrays with placeholder content:

In [6]:
# create a 2-dimensional array full of zeros with shape (2,3)
a = mx.nd.zeros((2,3))
# create a same shape array full of ones
b = mx.nd.ones((2,3))
# create a same shape array with all elements set to 7
c = mx.nd.full((2,3), 7)
# create a same shape whose initial content is random and
# depends on the state of the memory
d = mx.nd.empty((2,3))

e=mx.nd.random.randn(2,3)

## Printing Arrays

When inspecting the contents of an `NDArray`, it's often convenient to first
extract its contents as a `numpy.ndarray` using the `asnumpy` function.  Numpy
uses the following layout:

- The last axis is printed from left to right,
- The second-to-last is printed from top to bottom,
- The rest are also printed from top to bottom, with each slice separated from
  the next by an empty line.

In [7]:
b = mx.nd.arange(18).reshape((3,2,3))
b.asnumpy()

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.]],

       [[ 6.,  7.,  8.],
        [ 9., 10., 11.]],

       [[12., 13., 14.],
        [15., 16., 17.]]], dtype=float32)

## Basic Operations

When applied to NDArrays, the standard arithmetic operators apply *elementwise*
calculations. The returned value is a new array whose content contains the
result.

In [8]:
a = mx.nd.ones((2,3))
b = mx.nd.ones((2,3))
# elementwise plus
c = a + b
# elementwise minus
d = - c
# elementwise pow and sin, and then transpose
e = mx.nd.sin(c**2).T
# elementwise max
f = mx.nd.maximum(a, c)
f.asnumpy()

array([[2., 2., 2.],
       [2., 2., 2.]], dtype=float32)

In [11]:
mx.nd.random.gamma(alpha=1,beta=0.5,shape=(10,))


[0.8243321  0.38484073 0.3921469  0.10635482 0.6195445  0.02108932
 0.95644474 0.03200158 0.6266691  0.16187486]
<NDArray 10 @cpu(0)>

As in `NumPy`, `*` represents element-wise multiplication. For matrix-matrix
multiplication, use `dot`.

In [12]:
a = mx.nd.arange(4).reshape((2,2))
b = a * a
c = mx.nd.dot(a,a)
print("b: %s, \n c: %s" % (b.asnumpy(), c.asnumpy()))

b: [[0. 1.]
 [4. 9.]], 
 c: [[ 2.  3.]
 [ 6. 11.]]


The assignment operators such as `+=` and `*=` modify arrays in place, and thus
don't allocate new memory to create a new array.

In [13]:
a = mx.nd.ones((2,2))
b = mx.nd.ones(a.shape)
b=mx.nd.ones_like(a)
b += a
b.asnumpy()

array([[2., 2.],
       [2., 2.]], dtype=float32)

## Indexing and Slicing

The slice operator `[]` applies on axis 0.

In [14]:
a = mx.nd.array(np.arange(6).reshape(3,2))
a[1:2] = 1
a[:].asnumpy()

array([[0., 1.],
       [1., 1.],
       [4., 5.]], dtype=float32)

We can also slice a particular axis with the method `slice_axis`

In [19]:
# d = mx.nd.slice_axis(a, axis=1, begin=1, end=2)#d[:,1:2]
d=mx.nd.slice_axis(a,axis=1,begin=1,end=2)
print(a[:,1:2])
d.asnumpy()


[[1.]
 [1.]
 [5.]]
<NDArray 3x1 @cpu(0)>


array([[1.],
       [1.],
       [5.]], dtype=float32)

## Shape Manipulation

Using `reshape`, we can manipulate any arrays shape as long as the size remains
unchanged.

In [20]:
a = mx.nd.array(np.arange(24))
b = a.reshape((2,3,4))
b.asnumpy()

array([[[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]],

       [[12., 13., 14., 15.],
        [16., 17., 18., 19.],
        [20., 21., 22., 23.]]], dtype=float32)

The `concat` method stacks multiple arrays along the first axis. Their
shapes must be the same along the other axes.

In [30]:
a = mx.nd.ones((2,3))
b = mx.nd.ones((2,3))*2
c = mx.nd.concat(a,b,dim=-1) #注意这里是dim不是axis
c.asnumpy()

array([[1., 1., 1., 2., 2., 2.],
       [1., 1., 1., 2., 2., 2.]], dtype=float32)

## Reduce

Some functions, like `sum` and `mean` reduce arrays to scalars.

In [31]:
a = mx.nd.ones((2,3))
b = mx.nd.sum(a)
b.asnumpy()

array([6.], dtype=float32)

We can also reduce an array along a particular axis:

In [34]:
c = mx.nd.sum_axis(a, axis=1)
c.asnumpy()

array([3., 3.], dtype=float32)

## Broadcast

We can also broadcast an array. Broadcasting operations, duplicate an array's
value along an axis with length 1. The following code broadcasts along axis 1:

In [39]:
a = mx.nd.array(np.arange(6).reshape(6,1))
b = a.broadcast_to((6,3))  #
b.asnumpy()

array([[0., 0., 0.],
       [1., 1., 1.],
       [2., 2., 2.],
       [3., 3., 3.],
       [4., 4., 4.],
       [5., 5., 5.]], dtype=float32)

It's possible to simultaneously broadcast along multiple axes. In the following example, we broadcast along axes 1 and 2:

In [40]:
c = a.reshape((2,1,1,3))
d = c.broadcast_to((2,2,2,3))
d.asnumpy()

array([[[[0., 1., 2.],
         [0., 1., 2.]],

        [[0., 1., 2.],
         [0., 1., 2.]]],


       [[[3., 4., 5.],
         [3., 4., 5.]],

        [[3., 4., 5.],
         [3., 4., 5.]]]], dtype=float32)

Broadcasting can be applied automatically when executing some operations,
e.g. `*` and `+` on arrays of different shapes.

In [46]:
a = mx.nd.ones((3,2))
b = mx.nd.ones((1,2))
# b=b.broadcast_to((3,2))
c = a + b
c.asnumpy()

array([[2., 2.],
       [2., 2.],
       [2., 2.]], dtype=float32)

## Copies

When assigning an NDArray to another Python variable, we copy a reference to the
*same* NDArray. However, we often need to make a copy of the data, so that we
can manipulate the new array without overwriting the original values.

In [47]:
a = mx.nd.ones((2,2))
b = a
b is a # will be True

True

The `copy` method makes a deep copy of the array and its data:

In [48]:
b = a.copy()
b is a  # will be False

False

The above code allocates a new NDArray and then assigns to *b*. When we do not
want to allocate additional memory, we can use the `copyto` method or the slice
operator `[]` instead.

In [49]:
b = mx.nd.ones(a.shape)
c = b
c[:] = a
d = b
a.copyto(d)
(c is b, d is b)  # Both will be True

(True, True)

## Advanced Topics

MXNet's NDArray offers some advanced features that differentiate it from the
offerings you'll find in most other libraries.

### GPU Support

By default, NDArray operators are executed on CPU. But with MXNet, it's easy to
switch to another computation resource, such as GPU, when available. Each
NDArray's device information is stored in `ndarray.context`. When MXNet is
compiled with flag `USE_CUDA=1` and the machine has at least one NVIDIA GPU, we
can cause all computations to run on GPU 0 by using context `mx.gpu(0)`, or
simply `mx.gpu()`. When we have access to two or more GPUs, the 2nd GPU is
represented by `mx.gpu(1)`, etc.

**Note** In order to execute the following section on a cpu set gpu_device to mx.cpu().

In [50]:
gpu_device=mx.gpu() # Change this to mx.cpu() in absence of GPUs.


def f():
    a = mx.nd.ones((100,100))
    b = mx.nd.ones((100,100))
    c = a + b
    print(c)
# in default mx.cpu() is used
f()
# change the default context to the first GPU
with mx.Context(gpu_device):
    f()


[[2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 ...
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]]
<NDArray 100x100 @cpu(0)>


MXNetError: [11:43:17] ../src/imperative/imperative.cc:81: Operator _ones is not implemented for GPU.
Stack trace:
  [bt] (0) /home/zhangxk/AIProject/incubator-mxnet/python/mxnet/../../build/libmxnet.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x32) [0x7f2791183b62]
  [bt] (1) /home/zhangxk/AIProject/incubator-mxnet/python/mxnet/../../build/libmxnet.so(mxnet::Imperative::InvokeOp(mxnet::Context const&, nnvm::NodeAttrs const&, std::vector<mxnet::NDArray*, std::allocator<mxnet::NDArray*> > const&, std::vector<mxnet::NDArray*, std::allocator<mxnet::NDArray*> > const&, std::vector<mxnet::OpReqType, std::allocator<mxnet::OpReqType> > const&, mxnet::DispatchMode, mxnet::OpStatePtr)+0xace) [0x7f27912eeb6e]
  [bt] (2) /home/zhangxk/AIProject/incubator-mxnet/python/mxnet/../../build/libmxnet.so(mxnet::Imperative::Invoke(mxnet::Context const&, nnvm::NodeAttrs const&, std::vector<mxnet::NDArray*, std::allocator<mxnet::NDArray*> > const&, std::vector<mxnet::NDArray*, std::allocator<mxnet::NDArray

We can also explicitly specify the context when creating an array:

In [ ]:
a = mx.nd.ones((100, 100), gpu_device)
a

Currently, MXNet requires two arrays to sit on the same device for
computation. There are several methods for copying data between devices.

In [ ]:
a = mx.nd.ones((100,100), mx.cpu())
b = mx.nd.ones((100,100), gpu_device)
c = mx.nd.ones((100,100), gpu_device)
a.copyto(c)  # copy from CPU to GPU
d = b + c
e = b.as_in_context(c.context) + c  # same to above
{'d':d, 'e':e}

### Serialize From/To (Distributed) Filesystems

MXNet offers two simple ways to save (load) data to (from) disk. The first way
is to use `pickle`, as you might with any other Python objects. `NDArray` is
pickle-compatible.

In [51]:
import pickle as pkl
a = mx.nd.ones((2, 3))
# pack and then dump into disk
data = pkl.dumps(a)
pkl.dump(data, open('tmp.pickle', 'wb'))
# load from disk and then unpack
data = pkl.load(open('tmp.pickle', 'rb'))
b = pkl.loads(data)
b.asnumpy()

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

The second way is to directly dump to disk in binary format by using the `save`
and `load` methods. We can save/load a single NDArray, or a list of NDArrays:

In [52]:
a = mx.nd.ones((2,3))
b = mx.nd.ones((5,6))
mx.nd.save("temp.ndarray", [a,b])
c = mx.nd.load("temp.ndarray")
c

[
 [[1. 1. 1.]
  [1. 1. 1.]]
 <NDArray 2x3 @cpu(0)>, 
 [[1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]]
 <NDArray 5x6 @cpu(0)>]

It's also possible to save or load a dict of NDArrays in this fashion:

In [53]:
d = {'a':a, 'b':b}
mx.nd.save('temp.ndarray',d)
c = mx.nd.load("temp.ndarray")
c

{'a': 
 [[1. 1. 1.]
  [1. 1. 1.]]
 <NDArray 2x3 @cpu(0)>, 'b': 
 [[1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1. 1.]]
 <NDArray 5x6 @cpu(0)>}

The `load` and `save` methods are preferable to pickle in two respects

1. When using these methods, you can save data from within the Python interface
   and then use it later from another language's binding. For example, if we save
   the data in Python:

In [ ]:
a = mx.nd.ones((2, 3))
mx.nd.save("temp.ndarray", [a,])

we can later load it from R:
```
a <- mx.nd.load("temp.ndarray")
as.array(a[[1]])
##      [,1] [,2] [,3]
## [1,]    1    1    1
## [2,]    1    1    1
```

2. When a distributed filesystem such as Amazon S3 or Hadoop HDFS is set up, we
   can directly save to and load from it.

```
mx.nd.save('s3://mybucket/mydata.ndarray', [a,])  # if compiled with USE_S3=1
mx.nd.save('hdfs///users/myname/mydata.bin', [a,])  # if compiled with USE_HDFS=1
```

### Lazy Evaluation and Automatic Parallelization

MXNet uses lazy evaluation to achieve superior performance.  When we run `a=b+1`
in Python, the Python thread just pushes this operation into the backend engine
and then returns.  There are two benefits to this approach:

1. The main Python thread can continue to execute other computations once the
   previous one is pushed. It is useful for frontend languages with heavy
   overheads.
2. It is easier for the backend engine to explore further optimization, such as
   auto parallelization.

The backend engine can resolve data dependencies and schedule the computations
correctly. It is transparent to frontend users. We can explicitly call the
method `wait_to_read` on the result array to wait until the computation
finishes. Operations that copy data from an array to other packages, such as
`asnumpy`, will implicitly call `wait_to_read`.

In [ ]:
import time
def do(x, n):
    """push computation into the backend engine"""
    return [mx.nd.dot(x,x) for i in range(n)]
def wait(x):
    """wait until all results are available"""
    for y in x:
        y.wait_to_read()

tic = time.time()
a = mx.nd.ones((1000,1000))
b = do(a, 50)
print('time for all computations are pushed into the backend engine:\n %f sec' % (time.time() - tic))
wait(b)
print('time for all computations are finished:\n %f sec' % (time.time() - tic))

Besides analyzing data read and write dependencies, the backend engine is able
to schedule computations with no dependency in parallel. For example, in the
following code:

In [ ]:
a = mx.nd.ones((2,3))
b = a + 1
c = a + 2
d = b * c

the second and third lines can be executed in parallel. The following example
first runs on CPU and then on GPU:

In [ ]:
n = 10
a = mx.nd.ones((1000,1000))
b = mx.nd.ones((6000,6000), gpu_device)
tic = time.time()
c = do(a, n)
wait(c)
print('Time to finish the CPU workload: %f sec' % (time.time() - tic))
d = do(b, n)
wait(d)
print('Time to finish both CPU/GPU workloads: %f sec' % (time.time() - tic))

Now we issue all workloads at the same time. The backend engine will try to
parallel the CPU and GPU computations.

In [ ]:
tic = time.time()
c = do(a, n)
d = do(b, n)
wait(c)
wait(d)
print('Both as finished in: %f sec' % (time.time() - tic))


<!-- INSERT SOURCE DOWNLOAD BUTTONS -->

